## My Snoring 

Use the data of the sound snoring sensor and visualise it here. 

- Use snoring sensor on RPi and store it on the InfluxDB
- Use Grafana (hass.io) to get the data from the device.
- Download CSV from Grafana and import it here.

## Contents
0. Get the data and load into Pandas df
1. Data inspection, cleaning and visualisation
2. Statistical analysis
3. Store the results as json

## 0. Get the data and load into Pandas df

### 0a. The old fashioned way: using a .csv

In [33]:
pwd

'C:\\Users\\31653\\Documents\\GitHub\\Notebooks'

In [34]:
import glob
my_csvs = glob.glob('*.csv')
my_csvs

['2021-12-5_Features_.csv',
 'foo.csv',
 'OEtest_210505_youtube_traffic.csv',
 'OEtest_210506_youtubetraffic.csv',
 'Snoring-data-2021-12-03 09_15_24.csv',
 'Snoring-data-2021-12-04 11_41_48.csv',
 'Snoring-data-2021-12-04 11_51_51.csv',
 'Snoring-data-2021-12-06 19_03_36.csv',
 'Snoring-data-2021-12-10 13_27_41.csv',
 'Snoring-data-2021-12-12 09_38_24.csv',
 'snoring_2021_12_3-4b.csv',
 'UMTMVS.csv']

In [36]:
#import pandas as pd
# snoring_df=pd.read_csv('Snoring-data-2021-12-12 09_38_24.csv')
#snoring_df.tail()

Time  my_snoring.mean
43196  2021-12-12 09:37:36           0.0636
43197  2021-12-12 09:37:37           0.0627
43198  2021-12-12 09:37:38           0.0657
43199  2021-12-12 09:37:39           0.0671
43200  2021-12-12 09:37:40           0.0653

### 0b. The modern way: getting data directly from InfluxDB on Raspberry

In [4]:
# instantiate a client and assign the RPI and database to it and test the connection
import pandas as pd
from influxdb import DataFrameClient
import notebooks_config
mydf_client = DataFrameClient(host= notebooks_config.red_rpi_ip, port=8086, database='SNORING')# this is my raspberry ip on a local network
pang = mydf_client.ping()
print(pang)

1.8.10


In [29]:
#getting start and end time based on oura ring
from oura import OuraClient
# get your credentials from oura, see: https://cloud.ouraring.com/docs/.
# You will need a client_id, client_secret and a personal access token (PAT). 

import notebooks_config
#I store all my credentials in a notebooks_config.py file. 
#This file is then included in the .gitignore list so I can share the notebooks on GitHub.  

oura_pat = notebooks_config.oura_pat #
oura_client_id = notebooks_config.oura_client_id
oura_client_secret = notebooks_config.oura_client_secret
client = OuraClient(personal_access_token=oura_pat)
oura = OuraClient(oura_client_id, oura_client_secret, oura_pat)
result = oura.sleep_summary(start='2021-12-15', end='2021-12-16')
data = result['sleep'][0] #select the first item from the list
start_time = data['bedtime_start']
end_time = data['bedtime_end']
# convert strings to datetime
from dateutil import parser
start_time_dt = parser.parse(start_time)
end_time_dt = parser.parse(end_time)
from datetime import datetime
start_time_dt_ntz = start_time_dt.replace(tzinfo=None)
end_time_dt_ntz = end_time_dt.replace(tzinfo=None)
print ('start time is '+ str(start_time_dt_ntz) + ' and end time is ' + str(end_time_dt_ntz))

start time is 2021-12-15 22:59:39 and end time is 2021-12-16 06:50:39


In [27]:
#create query string for InfluxDB
base_string = "SELECT * FROM my_snoring WHERE time >= '"
middle_string = "' AND time <='"
end_string = "'"
query_string = base_string + str(start_time_dt_ntz) + middle_string + str(end_time_dt_ntz)+end_string
query_string

"SELECT * FROM my_snoring WHERE time >= '2021-12-15 22:59:39' AND time <='2021-12-16 06:50:39'"

In [28]:
#create a df and get the data from InfluxDB based on query above.
helper_df = mydf_client.query(query_string)
snoring_df =helper_df['my_snoring']
snoring_df

sheets   silence   snoring  softtalking
2021-12-15 22:59:39.058585+00:00  0.003854  0.846629  0.149485     0.000032
2021-12-15 22:59:39.603037+00:00  0.002465  0.926920  0.070599     0.000017
2021-12-15 22:59:40.146748+00:00  0.001982  0.932507  0.065498     0.000013
2021-12-15 22:59:40.686482+00:00  0.002001  0.931815  0.066171     0.000013
2021-12-15 22:59:41.225180+00:00  0.001969  0.931664  0.066354     0.000013
...                                    ...       ...       ...          ...
2021-12-16 06:50:36.744098+00:00  0.001949  0.932819  0.065220     0.000013
2021-12-16 06:50:37.288573+00:00  0.001953  0.932499  0.065535     0.000012
2021-12-16 06:50:37.834138+00:00  0.001904  0.933731  0.064353     0.000012
2021-12-16 06:50:38.377072+00:00  0.002001  0.931862  0.066124     0.000013
2021-12-16 06:50:38.918020+00:00  0.002003  0.931798  0.066187     0.000013

[52130 rows x 4 columns]

## 1. Data inspection, cleaning and visualisation


In [9]:
#show colums
snoring_df.columns

Index(['sheets', 'silence', 'snoring', 'softtalking'], dtype='object')

In [10]:
#describe data
snoring_df.describe()

sheets       silence       snoring   softtalking
count  5.069200e+04  50692.000000  50692.000000  5.069200e+04
mean   2.138438e-03      0.902277      0.095563  2.167598e-05
std    2.848076e-03      0.112315      0.112107  8.655829e-05
min    3.181834e-14      0.001169      0.031998  3.618848e-07
25%    1.887369e-03      0.931532      0.065544  1.242124e-05
50%    1.965800e-03      0.931865      0.066120  1.286520e-05
75%    2.001647e-03      0.932516      0.066532  1.300977e-05
max    2.886261e-01      0.967890      0.998828  6.650117e-03

## 2. Statistical analysis

In [11]:
# add a column 'bins' to the df and create bins with labels

snoring_df['bins'] = pd.cut(snoring_df['snoring'], bins=[0.0, 0.10, 0.50, 1.00],
                    labels=['no snoring', 'light snoring', 'loud snoring']) 


In [12]:
snoring_df.tail()

sheets   silence   snoring  softtalking  \
2021-12-16 06:37:37.089527+00:00  0.001999  0.931870  0.066118     0.000013   
2021-12-16 06:37:37.634071+00:00  0.001999  0.931788  0.066200     0.000013   
2021-12-16 06:37:38.174966+00:00  0.002003  0.931790  0.066195     0.000013   
2021-12-16 06:37:38.714911+00:00  0.001997  0.931850  0.066140     0.000013   
2021-12-16 06:37:39.251675+00:00  0.002002  0.931798  0.066187     0.000013   

                                        bins  
2021-12-16 06:37:37.089527+00:00  no snoring  
2021-12-16 06:37:37.634071+00:00  no snoring  
2021-12-16 06:37:38.174966+00:00  no snoring  
2021-12-16 06:37:38.714911+00:00  no snoring  
2021-12-16 06:37:39.251675+00:00  no snoring

In [13]:
total_length = snoring_df['bins'].count()
total_length

50692

In [14]:
stats_df = pd.DataFrame()
stats_df['class'] =snoring_df['bins'].value_counts()
stats_df

class
no snoring     44690
light snoring   4857
loud snoring    1145

In [34]:
#define parameters
no_snoring =stats_df['class'][0]
light_snoring=stats_df['class'][1]
loud_snoring=stats_df['class'][2]
#print results
time_snoring = round(((light_snoring+loud_snoring)/60),2)
print("You've snored " +str(time_snoring) + " minutes")
percentage = ((loud_snoring+light_snoring) / total_length)*100
print ("Total snoring was " + str(round(percentage,2)) + "% the measured time")
loud_perc = round(loud_snoring/total_length*100,2)
print ("Loud snoring was " + str(loud_perc) + "% of the measured time")

You've snored 100.03 minutes
Total snoring was 11.84% the measured time
Loud snoring was 2.26% of the measured time


### 2b. Make a graph

In [ ]:
#visualise the data. To do: larger plot and mean.values
#snoring_df.plot.scatter(x="Time", y="my_snoring.mean")


## 3. Store the results